In [3]:
import pandas as pd
import numpy as np

In [108]:

def getOverlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0]))




def get_utrs(df):
    if df.Strand.values[0] == "+":
        strand = "+"
        df = df.sort_values(by=['Start', 'End'])

    elif df.Strand.values[0] == "-":
        strand = "-"
        df = df.sort_values(by=['Start', 'End'], ascending=False)
        
    cds = df.loc[df['Type'] == "CDS", 'ranges'].values.tolist()
    exons = df.loc[df['Type'] == "exon", 'ranges'].values.tolist()
    diff_exons = sorted([int(sub_e) for e in list(set(exons).difference(set(cds))) for sub_e in e.split('-')])
    diff_exons = ["{}-{}".format(diff_exons[i], diff_exons[i+1]) for i, e in enumerate(diff_exons) if i % 2 == 0 and i < len(diff_exons) - 1]

    empty_dict = dict()
    utr_dict = dict()
    shared_five_dict = dict()
    shared_three_dict = dict()
    for j, e in enumerate(diff_exons):
        e = [int(sub_e) for sub_e in e.split('-')]
        for i, c in enumerate(cds):
            c = [int(sub_c) for sub_c in c.split('-')]
            if getOverlap(e,c) > 0:
                if i == 0:

                    if strand == "+":
                        new_e = [e[0], c[0] - 1]
                        utr_dict["5_prime"] = c[0] - 1 
                    elif strand == "-":
                        new_e = [c[1] + 1, e[1]]
                        utr_dict["5_prime"] = c[1] + 1 
                    shared_five_dict["-".join([str(sub_e) for sub_e in e])] = True
                    empty_dict["-".join([str(sub_e) for sub_e in e])] = "-".join([str(sub_e) for sub_e in new_e])
                    
                    
                elif i == len(cds) - 1:
                    if strand == "+":
                        new_e = [c[1] + 1, e[1]]
                        utr_dict["3_prime"] = c[1] + 1 
                        
                    elif strand == "-":
                        new_e = [e[0], c[0] - 1]  
                        utr_dict["3_prime"] = c[0] - 1 
                    shared_three_dict["-".join([str(sub_e) for sub_e in e])] = True
                    empty_dict["-".join([str(sub_e) for sub_e in e])] = "-".join([str(sub_e) for sub_e in new_e])
                    
        
    df['UTR_ranges'] = df['ranges'].map(empty_dict)
    df['Shared_5_prime'] = df['ranges'].map(shared_five_dict)
    df['Shared_5_prime'] = df['Shared_5_prime'].fillna(False)
    df['Shared_3_prime'] = df['ranges'].map(shared_three_dict)
    df['Shared_3_prime'] = df['Shared_3_prime'].fillna(False)    
#     print(empty_dict)
#     print(utr_dict)
#     print(shared_dict)

    df.loc[df['Type'] == 'CDS', 'Nb_exon'] = list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))
    df.loc[df['Type'] == 'exon', 'Nb_exon'] = list(range(1,df.loc[df['Type'] == 'exon'].shape[0] + 1))
    df.loc[df['Type'] == 'exon', 'Nb_exon_inverted'] = np.array(list(range(1,df.loc[df['Type'] == 'exon'].shape[0] + 1))[::-1]) * -1
    df.loc[df['Type'] == 'CDS', 'Nb_exon_inverted'] = np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))[::-1]) * -1

#     print(strand)
#     print(utr_dict)
#     print(empty_dict

#     try:

    if strand == "+":

        if '5_prime' in utr_dict:
            df.loc[df['End'] < utr_dict['5_prime'], 'UTR_ranges'] = df.loc[df['End'] < utr_dict['5_prime']]['ranges']
            df.loc[df['Start'] < utr_dict['5_prime'], 'UTR_bool'] = "5_prime"
            if df.loc[(df['UTR_bool'] == '5_prime') & (df['Shared_5_prime'] == False)].empty is False:
                df.loc[df['Type'] == 'CDS', 'Nb_exon_relative'] = np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))) + df.loc[(df['UTR_bool'] == '5_prime') & (df['Shared_5_prime'] == False)]['Nb_exon'].values[-1] 
            else:
                df.loc[df['Type'] == 'CDS', 'Nb_exon_relative'] = np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))) 

        if '3_prime' in utr_dict:
            df.loc[df['Start'] > utr_dict['3_prime'], 'UTR_ranges'] = df.loc[df['Start'] > utr_dict['3_prime']]['ranges']
            df.loc[df['End'] > utr_dict['3_prime'], 'UTR_bool'] = "3_prime"

    elif strand == "-":

        if '5_prime' in utr_dict:
            df.loc[df['End'] > utr_dict['5_prime'], 'UTR_bool'] = "5_prime"
            df.loc[df['Start'] > utr_dict['5_prime'], 'UTR_ranges'] = df.loc[df['End'] > utr_dict['5_prime']]['ranges']
#                 df.loc[df['Type'] == 'CDS', 'Nb_exon_relative'] = np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))) + df.loc[df['UTR_bool'] == '5_prime']['Nb_exon'].values[-1] 
            if df.loc[(df['UTR_bool'] == '5_prime') & (df['Shared_5_prime'] == False)].empty is False:
                df.loc[df['Type'] == 'CDS', 'Nb_exon_relative'] = np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))) + df.loc[(df['UTR_bool'] == '5_prime') & (df['Shared_5_prime'] == False)]['Nb_exon'].values[-1] 
            else:
                df.loc[df['Type'] == 'CDS', 'Nb_exon_relative'] = np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))) 

        if '3_prime' in utr_dict:
            df.loc[df['End'] < utr_dict['3_prime'], 'UTR_ranges'] = df.loc[df['Start'] < utr_dict['3_prime']]['ranges']
            df.loc[df['Start'] < utr_dict['3_prime'], 'UTR_bool'] = "3_prime"

            
#     except: 
#         print(df)

#     df['UTR_length'] = df.apply(lambda r: [int(e) for e in r['UTR_ranges'].split('-')][1] - [int(e) for e in r['UTR_ranges'].split('-')][0], axis=1)

    return df



In [142]:
def process_introns_utr(df):
#     df['Strand'] = '+'
    if df.Strand.values[0] == "+":
        strand = "+"
        ascending=True
        reverse=False
        df = df.sort_values(by=['Start', 'End'])

    elif df.Strand.values[0] == "-":
        strand = "-"
        ascending=False
        reverse=True
        df = df.sort_values(by=['Start', 'End'], ascending=ascending)
    df.loc[df['Type'] == 'CDS', 'Nb_exon'] = list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))
    df.loc[df['Type'] == 'exon', 'Nb_exon'] = list(range(1,df.loc[df['Type'] == 'exon'].shape[0] + 1))
    introns_exons = df.loc[df['Type'] == 'exon'].ranges.values.tolist()
    introns_cds = df.loc[df['Type'] == 'CDS'].ranges.values.tolist()
#     print(df.Gene.values[0], df.NM.values[0])
#     print(introns_exons)
#     print(introns_cds)
    diff_exons = sorted(set(introns_exons).difference(set(introns_cds)), reverse=reverse)
    print(diff_exons)
    if ascending is True:
        first_cds_start = int(introns_cds[0].split('-')[0])
        print(first_cds_start)
#         print(first_cds_start)
        diff_exons = [e for e in diff_exons if int(e.split('-')[1]) < first_cds_start]
#         relative_nb = len(diff_exons)
        
    elif ascending is False:
        first_cds_start = int(introns_cds[0].split('-')[1])
        diff_exons = [e for e in diff_exons if int(e.split('-')[0]) > first_cds_start]
    print(diff_exons)
    relative_nb = len(diff_exons)

    print(df)
    print(np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))))
    print(relative_nb)
    df.loc[df['Type'] == 'CDS', 'Nb_exon_relative'] = np.array(list(range(1,df.loc[df['Type'] == 'CDS'].shape[0] + 1))) + relative_nb

#     print(diff_exons)
#     print('\n')
    return df 

In [143]:
test_df_cds = pd.DataFrame([


    {
        'Start' : 10,
        'End' : 15,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },    
   {
        'Start' : 20,
        'End' : 25,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },    
])
test_df_cds['ranges'] = test_df_cds['Start'].astype(str) + '-' + test_df_cds['End'].astype(str)
test_df_cds['Type'] = 'CDS'
test_df_cds = test_df_cds.sort_values(by=['Gene', 'mRNA', 'Start'])

test_df_exon = pd.DataFrame([

    {
        'Start' : 0,
        'End' : 3,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },
    {
        'Start' : 10,
        'End' : 15,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },    
   {
        'Start' : 20,
        'End' : 25,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },  
 
])
test_df_exon['ranges'] = test_df_exon['Start'].astype(str) + '-' + test_df_exon['End'].astype(str)
test_df_exon['Type'] = 'exon'

concat_cds_exon = pd.concat(
    [test_df_cds, test_df_exon]
)
concat_cds_exon['Strand'] = '-

'
concat_cds_exon = concat_cds_exon.rename({'mRNA' : 'NM'}, axis=1)
concat_cds_exon

concat_cds_exon = concat_cds_exon.groupby(['Gene', 'NM']).apply(lambda df: process_introns_utr(df))
concat_cds_exon

['0-3']
10
['0-3']
   Start  End ID Gene NM ranges  Type Strand  Nb_exon
0      0    3  A  AAA  1    0-3  exon      +      1.0
0     10   15  A  AAA  1  10-15   CDS      +      1.0
1     10   15  A  AAA  1  10-15  exon      +      2.0
1     20   25  A  AAA  1  20-25   CDS      +      2.0
2     20   25  A  AAA  1  20-25  exon      +      3.0
[1 2]
1


Start  End ID Gene NM ranges  Type Strand  Nb_exon  \
Gene NM                                                         
AAA  1  0      0    3  A  AAA  1    0-3  exon      +      1.0   
        0     10   15  A  AAA  1  10-15   CDS      +      1.0   
        1     10   15  A  AAA  1  10-15  exon      +      2.0   
        1     20   25  A  AAA  1  20-25   CDS      +      2.0   
        2     20   25  A  AAA  1  20-25  exon      +      3.0   

           Nb_exon_relative  
Gene NM                      
AAA  1  0               NaN  
        0               2.0  
        1               NaN  
        1               3.0  
        2               NaN

In [129]:
test_df_cds = pd.DataFrame([

    {
        'Start' : 7,
        'End' : 8,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },
    {
        'Start' : 10,
        'End' : 13,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },    
    {
        'Start' : 8,
        'End' : 10,
        'ID' : 'B',
        'Gene' : 'AAA',
        'mRNA' : '2',
    },
    
    {
        'Start' : 18,
        'End' : 20,
        'ID' : 'D',
        'Gene' : 'AAA',        
        'mRNA' : '1',        
    },    
    {
        'Start' : 20,
        'End' : 22,
        'ID' : 'D',
        'Gene' : 'AAA',        
        'mRNA' : '2',        
    },        
        {
        'Start' : 18,
        'End' : 20,
        'ID' : 'D',
        'Gene' : 'BBB',        
        'mRNA' : '1',        
    },    
    {
        'Start' : 20,
        'End' : 22,
        'ID' : 'D',
        'Gene' : 'BBB',        
        'mRNA' : '1',        
    },  
#     {
#         'Start' : 7,
#         'End' : 8,
#         'ID' : 'D',
#         'Gene' : 'AAA',        
#     },    
#     {
#         'Start' : 3,
#         'End' : 4,
#         'ID' : 'D',
#         'Gene' : 'BBB',       
#         'mRNA' : '1',        
        
#     },
#     {
#         'Start' : 10,
#         'End' : 15,
#         'ID' : 'D',
#         'Gene' : 'BBB',     
#         'mRNA' : '1',        
        
#     },    
])
test_df_cds['ranges'] = test_df_cds['Start'].astype(str) + '-' + test_df_cds['End'].astype(str)
test_df_cds['Type'] = 'CDS'
test_df_cds = test_df_cds.sort_values(by=['Gene', 'mRNA', 'Start'])

test_df_exon = pd.DataFrame([

    {
        'Start' : 5,
        'End' : 8,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },
    {
        'Start' : 0,
        'End' : 1,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },
    {
        'Start' : 3,
        'End' : 3,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },
    {
        'Start' : 10,
        'End' : 13,
        'ID' : 'A',
        'Gene' : 'AAA',
        'mRNA' : '1',
    },    
    {
        'Start' : 1,
        'End' : 10,
        'ID' : 'B',
        'Gene' : 'AAA',
        'mRNA' : '2',
    },
    
    {
        'Start' : 18,
        'End' : 26,
        'ID' : 'D',
        'Gene' : 'AAA',        
        'mRNA' : '1',        
    },    
    {
        'Start' : 20,
        'End' : 26,
        'ID' : 'D',
        'Gene' : 'AAA',        
        'mRNA' : '2',        
    }, 
    {
        'Start' : 30,
        'End' : 36,
        'ID' : 'D',
        'Gene' : 'AAA',        
        'mRNA' : '2',        
    },  
        {
        'Start' : 18,
        'End' : 20,
        'ID' : 'D',
        'Gene' : 'BBB',        
        'mRNA' : '1',        
    },    
    {
        'Start' : 20,
        'End' : 22,
        'ID' : 'D',
        'Gene' : 'BBB',        
        'mRNA' : '1',        
    },     
#     {
#         'Start' : 7,
#         'End' : 8,
#         'ID' : 'D',
#         'Gene' : 'AAA',        
#     },    
#     {
#         'Start' : 3,
#         'End' : 4,
#         'ID' : 'D',
#         'Gene' : 'BBB',       
#         'mRNA' : '1',        
        
#     },
#     {
#         'Start' : 10,
#         'End' : 15,
#         'ID' : 'D',
#         'Gene' : 'BBB',     
#         'mRNA' : '1',        
        
#     },    
])
test_df_exon['ranges'] = test_df_exon['Start'].astype(str) + '-' + test_df_exon['End'].astype(str)
test_df_exon['Type'] = 'exon'

concat_cds_exon = pd.concat(
    [test_df_cds, test_df_exon]
)
concat_cds_exon['Strand'] = '+'
concat_cds_exon = concat_cds_exon.rename({'mRNA' : 'NM'}, axis=1)
concat_cds_exon

concat_cds_exon = concat_cds_exon.groupby(['Gene', 'NM']).apply(lambda df: get_utrs(df))
concat_cds_exon

Start  End ID Gene NM ranges  Type Strand UTR_ranges  \
Gene NM                                                           
AAA  1  1      0    1  A  AAA  1    0-1  exon      +        0-1   
        2      3    3  A  AAA  1    3-3  exon      +        3-3   
        0      5    8  A  AAA  1    5-8  exon      +        5-6   
        0      7    8  A  AAA  1    7-8   CDS      +        NaN   
        1     10   13  A  AAA  1  10-13   CDS      +        NaN   
        3     10   13  A  AAA  1  10-13  exon      +        NaN   
        3     18   20  D  AAA  1  18-20   CDS      +        NaN   
        5     18   26  D  AAA  1  18-26  exon      +      21-26   
     2  4      1   10  B  AAA  2   1-10  exon      +        1-7   
        2      8   10  B  AAA  2   8-10   CDS      +        NaN   
        4     20   22  D  AAA  2  20-22   CDS      +        NaN   
        6     20   26  D  AAA  2  20-26  exon      +      23-26   
        7     30   36  D  AAA  2  30-36  exon      +      30-36   
BBB  1  5     18   20  D  BBB  1  18-20   CDS      +        NaN   
        8     18   20  D  BBB  1  18-20  exon      +        NaN   
        6     20   22  D  BBB  1  20-22   CDS      +        NaN   
        9     20   22  D  BBB  1  20-22  exon      +        NaN   

           Shared_5_prime  Shared_3_prime  Nb_exon  Nb_exon_inverted UTR_bool  \
Gene NM                                                                         
AAA  1  1           False           False      1.0              -5.0  5_prime   
        2           False           False      2.0              -4.0  5_prime   
        0            True           False      3.0              -3.0  5_prime   
        0           False           False      1.0              -3.0      NaN   
        1           False           False      2.0              -2.0      NaN   
        3           False           False      4.0              -2.0      NaN   
        3           False           False      3.0              -1.0      NaN   
        5           False            True      5.0              -1.0  3_prime   
     2  4            True           False      1.0              -3.0  5_prime   
        2           False           False      1.0              -2.0      NaN   
        4           False           False      2.0              -1.0      NaN   
        6           False            True      2.0              -2.0  3_prime   
        7           False           False      3.0              -1.0  3_prime   
BBB  1  5           False           False      1.0              -2.0      NaN   
        8           False           False      1.0              -2.0      NaN   
        6           False           False      2.0              -1.0      NaN   
        9           False           False      2.0              -1.0      NaN   

           Nb_exon_relative  
Gene NM                      
AAA  1  1               NaN  
        2               NaN  
        0               NaN  
        0               3.0  
        1               4.0  
        3               NaN  
        3               5.0  
        5               NaN  
     2  4               NaN  
        2               1.0  
        4               2.0  
        6               NaN  
        7               NaN  
BBB  1  5               NaN  
        8               NaN  
        6               NaN  
        9               NaN

In [111]:
print(concat_cds_exon.loc[concat_cds_exon['Nb_exon_relative'].isna() == True].index.tolist())


[('AAA', '1', 5), ('AAA', '1', 3), ('AAA', '1', 0), ('AAA', '1', 2), ('AAA', '1', 1), ('AAA', '2', 7), ('AAA', '2', 6), ('AAA', '2', 4), ('BBB', '1', 6), ('BBB', '1', 9), ('BBB', '1', 5), ('BBB', '1', 8)]


In [123]:
concat_cds_exon.loc[index_wt_relative]['Nb_exon_relative']

0    NaN
3    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
Name: Nb_exon_relative, dtype: float64

In [119]:
concat_cds_exon.loc[index_wt_relative]['Nb_exon']

0     1.0
3     2.0
5     3.0
6     4.0
7     5.0
8     1.0
9     2.0
12    3.0
13    1.0
14    1.0
15    2.0
16    2.0
Name: Nb_exon, dtype: float64

In [130]:
concat_cds_exon = concat_cds_exon.reset_index(drop=True)
index_wt_relative = concat_cds_exon.loc[concat_cds_exon['Nb_exon_relative'].isna() == True].index.tolist()
concat_cds_exon.loc[index_wt_relative, 'Nb_exon_relative'] = concat_cds_exon.loc[index_wt_relative, 'Nb_exon'].values.tolist()
concat_cds_exon

,Start,End,ID,Gene,NM,ranges,Type,Strand,UTR_ranges,Shared_5_prime,Shared_3_prime,Nb_exon,Nb_exon_inverted,UTR_bool,Nb_exon_relative
0,0,1,A,AAA,1,0-1,exon,+,0-1,False,False,1.0,-5.0,5_prime,1.0
1,3,3,A,AAA,1,3-3,exon,+,3-3,False,False,2.0,-4.0,5_prime,2.0
2,5,8,A,AAA,1,5-8,exon,+,5-6,True,False,3.0,-3.0,5_prime,3.0
3,7,8,A,AAA,1,7-8,CDS,+,NaN,False,False,1.0,-3.0,NaN,3.0
4,10,13,A,AAA,1,10-13,CDS,+,NaN,False,False,2.0,-2.0,NaN,4.0
5,10,13,A,AAA,1,10-13,exon,+,NaN,False,False,4.0,-2.0,NaN,4.0
6,18,20,D,AAA,1,18-20,CDS,+,NaN,False,False,3.0,-1.0,NaN,5.0
7,18,26,D,AAA,1,18-26,exon,+,21-26,False,True,5.0,-1.0,3_prime,5.0
8,1,10,B,AAA,2,1-10,exon,+,1-7,True,False,1.0,-3.0,5_prime,1.0
9,8,10,B,AAA,2,8-10,CDS,+,NaN,False,False,1.0,-2.0,NaN,1.0
